In [1]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

In [2]:
pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

In [3]:
%matplotlib notebook

In [4]:
df = pd.read_excel('赛题1数据材料/赛题1数据集/购地-房地产大企业购地情况.xlsx', index_col='企业编号')
df

,行政区,签订日期,供地总面积（公顷）,约定动工时间,土地用途,供应方式,最小容积率,最大容积率,成交价款（万元）,约定竣工时间
企业编号,,,,,,,,,,
1001,内蒙古->呼伦贝尔市->海拉尔区,2010-07-14,0.018960,NaN,商务金融用地,NaN,NaN,NaN,56.0000,NaN
1001,内蒙古->呼伦贝尔市->海拉尔区,2010-07-14,0.012060,NaN,商务金融用地,NaN,NaN,NaN,40.0000,NaN
1001,内蒙古->呼伦贝尔市->海拉尔区,2010-07-14,0.049570,NaN,商务金融用地,NaN,NaN,NaN,136.0000,NaN
1001,湖南省 > 长沙市 > 长沙市本级,2012-11-20,0.762853,NaN,商务金融用地,NaN,NaN,NaN,10299.0000,NaN
1001,湖南省 > 长沙市 > 岳麓区,2012-11-26,0.762853,NaN,商务金融用地,NaN,NaN,NaN,10299.0000,NaN
...,...,...,...,...,...,...,...,...,...,...
3995,广东省 > 深圳市 > 深圳市本级,2011-04-13,3.895493,NaN,工业用地,NaN,NaN,NaN,2842.3500,NaN
3998,鼓楼区,2015-10-08,0.010798,NaN,商务金融用地,NaN,NaN,NaN,156.2147,NaN
3998,江苏省->南京市本级,2007-09-20,0.670000,NaN,采矿用地,NaN,NaN,NaN,NaN,NaN


In [5]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

In [6]:
appointment_time = df[['约定动工时间']].applymap(to_day).groupby('企业编号').mean().T
appointment_time = appointment_time.reindex(columns=list(range(1001, 4001)))
appointment_time

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
约定动工时间,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2207.428571,-3498.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
dt = df[['约定竣工时间']].applymap(to_day).groupby('企业编号').mean().T
dt = dt.reindex(columns=list(range(1001, 4001)))
dt


企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
约定竣工时间,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df["最小容积率"].value_counts()

0.00    204
1.00    199
2.00     38
1.50      8
1.60      7
       ... 
2.15      1
3.04      1
2.10      1
3.50      1
1.34      1
Name: 最小容积率, Length: 27, dtype: int64

In [9]:
def limit(cell):
    if cell > 1:
        return 1
    return cell

In [11]:
dummy_1=pd.get_dummies(df["供应方式"],prefix="供应方式")
dummy_1=dummy_1.groupby("企业编号").sum().T
dummy_1=dummy_1.applymap(limit).reindex(columns=list(range(1001, 4001))).fillna(0)
dummy_1

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
供应方式_划拨,0,0.0,0,0,0,0.0,0,1,1,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
供应方式_协议出让,0,0.0,0,0,0,0.0,0,1,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
供应方式_拍卖出让,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
供应方式_招标出让,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0
供应方式_挂牌出让,0,0.0,0,0,0,0.0,0,0,0,0,...,0,0.0,0,0,0,0.0,0.0,0,0,0


In [11]:
m=df.groupby(["企业编号"]).sum().drop(columns=["供地总面积（公顷）","成交价款（万元）"]).T.reindex(columns=list(range(1001, 4001)))
m

企业编号,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,...,3991,3992,3993,3994,3995,3996,3997,3998,3999,4000
最小容积率,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0
最大容积率,0.0,NaN,0.0,0.0,0.0,NaN,0.0,0.0,0.0,0.0,...,0.0,NaN,0.0,0.0,0.0,NaN,NaN,0.0,0.0,0.0


In [12]:
appointment_time.append([dt,dummy_1,m]).to_pickle('购地-房地产大企业购地情况.pickle')